<a href="https://colab.research.google.com/github/SUYAMBULAKSHMI-VENKATESAN/DAMAKA/blob/main/IrisSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')  # ONLY mount to /content/drive

# Set path to your dataset folder
DATASET_DIR = "/content/drive/MyDrive/ConsolidatedImages"

# Optional test
import os
files = os.listdir(DATASET_DIR)
print("Total files:", len(files))

Mounted at /content/drive
Total files: 2251


**1. Install Prerequisites (if not already installed)**

In [20]:
!pip install einops
!pip install torchvision
!pip install timm  # For transformer backbone
!pip install scikit-learn

**2. Import necessary libraries**

In [19]:
import os
import random
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
import timm  # For Vision Transformers
from einops import rearrange
from sklearn.model_selection import train_test_split

import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt


**PreProcessing**

In [18]:
import cv2
import os
import numpy as np
import pandas as pd
from google.colab import drive
from tqdm import tqdm

# Mount Google Drive
drive.mount('/content/drive')

# Path to your dataset folder
dataset_path = '/content/drive/MyDrive/ConsolidatedImages'
output_path = '/content/drive/MyDrive/PreprocessedImages'
os.makedirs(output_path, exist_ok=True)

# Function: Preprocess image using Gaussian + CLAHE + HE
def preprocess_image(image):
    gaussian_blur = cv2.GaussianBlur(image, (5, 5), 1.0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gaussian_blur)
    he_img = cv2.equalizeHist(clahe_img)
    return he_img

# Function: Compute image stats
def compute_stats(img):
    return {
        'mean': round(np.mean(img), 2),
        'std_dev': round(np.std(img), 2),
        'pixel_count': img.size
    }

# Process all images and save stats
stats_list = []
image_files = [f for f in os.listdir(dataset_path) if f.lower().endswith('.bmp')]

for file in tqdm(image_files, desc="Preprocessing images"):
    input_path = os.path.join(dataset_path, file)
    img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    preprocessed_img = preprocess_image(img)
    output_img_path = os.path.join(output_path, file)
    cv2.imwrite(output_img_path, preprocessed_img)

    stats = compute_stats(preprocessed_img)
    stats['filename'] = file
    stats_list.append(stats)

# Save stats to CSV
stats_df = pd.DataFrame(stats_list)
stats_df.to_csv('/content/drive/MyDrive/image_preprocessing_stats.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Preprocessing images: 100%|██████████| 2251/2251 [01:31<00:00, 24.62it/s]


**3. Custom Dataset for Rotation Prediction**

In [28]:
PREPROCESSED_DIR = "/content/drive/MyDrive/PreprocessedImages"

# Optional test
import os
files = os.listdir(PREPROCESSED_DIR)
print("Total files:", len(files))

Total files: 2251


In [29]:
import os
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# ✅ Change this path to your actual preprocessed image folder in Google Drive
PREPROCESSED_DIR = "/content/drive/MyDrive/PreprocessedImages"

# ✅ Custom Dataset for Rotation Prediction
class RotationPredictionDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_paths = [os.path.join(image_dir, f)
                            for f in os.listdir(image_dir) if f.lower().endswith('.bmp')]
        self.transform = transform
        self.rotations = [0, 90, 180, 270]

    def __len__(self):
        return len(self.image_paths) * 4  # 4 rotations per image

    def __getitem__(self, idx):
        img_idx = idx // 4
        rot_idx = idx % 4

        image = Image.open(self.image_paths[img_idx]).convert("L")  # Grayscale
        image = image.rotate(self.rotations[rot_idx])

        label = rot_idx  # 0→0°, 1→90°, 2→180°, 3→270°

        if self.transform:
            image = self.transform(image)

        return image, label

# ✅ Transform for resizing and converting to tensor
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# ✅ Create Dataset and DataLoader
dataset = RotationPredictionDataset(DATASET_DIR, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# ✅ CNN Model for Rotation Prediction
class RotationNet(nn.Module):
    def __init__(self):
        super(RotationNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, 4)  # 4 rotation classes
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


**6. Train the SSL Model**

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model, loss function, and optimizer
model = RotationNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total * 100
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.2f}%")


# Optional: Save model after training
torch.save(model.state_dict(), "/content/drive/MyDrive/rotation_ssl_model.pth")

Epoch 1/10: 100%|██████████| 282/282 [06:17<00:00,  1.34s/it]


Epoch [1/10] - Loss: 0.0686 - Accuracy: 97.85%


Epoch 2/10: 100%|██████████| 282/282 [06:12<00:00,  1.32s/it]


Epoch [2/10] - Loss: 0.0063 - Accuracy: 99.79%


Epoch 3/10: 100%|██████████| 282/282 [06:36<00:00,  1.41s/it]


...

Epoch [8/10] - Loss: 0.0000 - Accuracy: 100.00%


Epoch 9/10: 100%|██████████| 282/282 [06:30<00:00,  1.38s/it]


Epoch [9/10] - Loss: 0.0000 - Accuracy: 100.00%


Epoch 10/10: 100%|██████████| 282/282 [06:37<00:00,  1.41s/it]

Epoch [10/10] - Loss: 0.0000 - Accuracy: 100.00%


In [8]:
# ✅ Upload the model manually (run only once)
from google.colab import files
uploaded = files.upload()  # Do NOT pass a path here




# ✅ Load the model from the correct path (in /content)
model = RotationNet()
model.load_state_dict(torch.load("rotation_ssl_model.pth"))  # not /mnt/data
model.eval()
model.to(device)

# ✅ Rebuild features and labels
sequences = []
labels = []

with torch.no_grad():
    for images, lbls in tqdm(dataloader, desc="Extracting Features"):
        images = images.to(device)
        features = model.features(images)                 # Shape: (B, 64, 32, 32)
        features = features.view(features.size(0), -1)    # Shape: (B, 65536)
        sequences.append(features.cpu())
        labels.extend(lbls.cpu().tolist())

# ✅ Stack everything
sequences = torch.cat(sequences, dim=0)                      # (N, 65536)
labels_tensor = torch.tensor(labels, dtype=torch.long)       # (N,)


KeyboardInterrupt: 

In [9]:
# all_features: shape (N, 64, 32, 32)
B, C, H, W = all_features.shape  # B=batch, C=64, H=W=32
sequences = all_features.permute(0, 2, 3, 1).reshape(B, H*W, C)  # (B, 1024, 64)

import torch.nn as nn

class CNNTransformer(nn.Module):
    def __init__(self, input_dim=64, num_heads=4, num_layers=2, num_classes=2):
        super(CNNTransformer, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=128, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)  # 2 classes: iris vs non-iris (segmentation/classification)
        )

    def forward(self, x):  # x: (B, 1024, 64)
        x = self.transformer(x)
        x = x.mean(dim=1)  # Global average pooling across sequence
        return self.classifier(x)


NameError: name 'all_features' is not defined

In [11]:
sequences = []
labels = []

with torch.no_grad():
    for images, lbls in tqdm(dataloader, desc="Extracting Features"):
        images = images.to(device)
        features = model.features(images)          # (B, 64, 32, 32)
        sequences.append(features.cpu())           # Don't flatten!
        labels.extend(lbls.cpu().tolist())

# This is the line you're currently missing:
all_features = torch.cat(sequences, dim=0)  # Final shape: (N, 64, 32, 32)
labels_tensor = torch.tensor(labels, dtype=torch.long)


NameError: name 'dataloader' is not defined

In [53]:
class TransformerFeatureDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features  # shape: (N, 1024, 64)
        self.labels = labels      # shape: (N,)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [54]:
labels_tensor = labels.clone().detach() if isinstance(labels, torch.Tensor) else torch.tensor(labels, dtype=torch.long)

transformer_dataset = TransformerFeatureDataset(sequences, labels_tensor)
transformer_loader = DataLoader(transformer_dataset, batch_size=32, shuffle=True)

In [61]:
from torch.utils.data import Dataset, DataLoader

class TransformerFeatureDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels.clone().detach() if isinstance(labels, torch.Tensor) else torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# ✅ Rebuild DataLoader cleanly
transformer_dataset = TransformerFeatureDataset(sequences, labels_tensor)
transformer_loader = DataLoader(transformer_dataset, batch_size=32, shuffle=True)


In [2]:
import torch.nn as nn

class TransformerClassifier(nn.Module):
    def __init__(self, input_dim=64, num_classes=4, num_heads=4, num_layers=2, hidden_dim=256):
        super(TransformerClassifier, self).__init__()

        self.embedding = nn.Linear(input_dim, hidden_dim)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):  # x: (batch_size, sequence_len, input_dim)
        x = self.embedding(x)      # (B, 1024, 256)
        x = self.transformer(x)    # (B, 1024, 256)
        x = x.mean(dim=1)          # (B, 256)
        return self.classifier(x)  # (B, num_classes)


In [4]:
print("Features shape:", sequences.shape)
print("Labels shape:", labels_tensor.shape)

min_len = min(len(sequences), len(labels_tensor))
sequences = sequences[:min_len]
labels_tensor = labels_tensor[:min_len]

NameError: name 'sequences' is not defined

In [1]:
model = TransformerClassifier(input_dim=64, num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for inputs, labels in tqdm(transformer_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = 100.0 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.2f}%")


NameError: name 'TransformerClassifier' is not defined